In [1]:
%load_ext dotenv
%dotenv

In [2]:
# from ynab_split.sheets.service import get_user_credentials
# get_user_credentials()

In [3]:
import yaml
from ynab_split.sync.config import SyncSettings

with open("../config.yml", "r") as f:
    settings = SyncSettings.model_validate(yaml.safe_load(f))

In [ ]:
from ynab_split.sync.service import get_all_transactions, sync
# await sync(settings)
await get_all_transactions(settings)

In [5]:
from splitwise import Splitwise

from ynab_split.splitwise.config import splitwise_client_settings

sw = Splitwise(
    splitwise_client_settings.consumer_key,
    splitwise_client_settings.consumer_secret,
    api_key=splitwise_client_settings.api_key,
)


In [ ]:
from ynab_split.splitwise.service import describe_groups
describe_groups(sw)



In [ ]:
from ynab_split.splitwise.service import create_group_expense
from datetime import date
import json

create_group_expense(
    sw, 
    group_id=77728222, 
    amount=103.57,
    date=date(2024, 1, 28), 
    description="Test expense", 
    details=json.dumps({
        "source": "ynab_split",
        "ynab_transaction_id": "test-ynab-transaction-id"
    }, indent=4)
)

In [32]:
from ynab_split.splitwise.service import get_group_expenses
df = get_group_expenses(sw, group_id=77728222)

In [ ]:
import polars as pl
def is_valid_json(s):
    try:
        json.loads(s)
        return True
    except Exception:
        return False


(
    df
    .filter(pl.col("details").map_elements(is_valid_json))
    .filter(
        pl.col("details").map_elements(lambda x: json.loads(x).get("source"))
        .eq("ynab_split"))
)